# Generate ATP Synthase with Proton Pump SBML files
Will try to incorporate it instead of using SBML 

---

In [2]:
from biocrnpyler import *
#from biocrnpyler.chemical_reaction_network import Species, Reaction, ComplexSpecies, Multimer
import numpy as np
import pylab as plt 
%matplotlib inline

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

import colorcet

Loading BokehJS ...

ATP synthase DNA --> ATP synthase RNA <br>
ATP synthase RNA --> ATP synthase protein (folding) <br>
ATP synthase protein --> ATP synthase membrane integration <br>
<br> 
ATP synthesis through ATP synthase protein <br>
ATP hydrolysis through ATP synthase protein <br>
Proton transport through ATP synthase protein <br>
<br>
Proton pump DNA --> Proton pump RNA <br>
Proton pump RNA --> Proton pump protein (folding) <br>
Proton pump protein --> Proton pump membrane integration <br>
<br>
Proton transport through proton pump

**SBML Planning** <br>
- One sbml file for ATP synthase membrane integration
- One sbml file for ATP synthase synthesis/hydrolysis/proton transport 
- One sbml file for proton pump membrane integration
- One sbml file for proton pump proton transport

## SBML for ATP transcription and translation
---

In [13]:

#promoter is the name of the promoter or a Promoter component
#RBS is the name of the RBS or an RBS Component
#   RBS = None means there will be no translation
#By default (if transcript and protein are None), the transgript and protein will have the same name as an assmebly
#   Users can also assign Species or String names to transcript and protein
G = DNAassembly("atpsyn", promoter = "atpsyn_promoter", rbs = "atpsyn_rbs", transcript = None, protein = None)
M3 = TxTlDilutionMixture("liposome", components = [G], parameter_file = "default_parameters.txt")
CRN3 = M3.compile_crn()


#print(repr(M3),"\n", CRN3.pretty_print(show_attributes = True, show_material = True, show_rates = True),"\n\n")


In [8]:
# Write SBML file
CRN3.write_sbml_file('atpsynthase_tx_tl.xml')

True

In [3]:
timepoints = np.linspace(0,10000,100)
x0 = {'dna_atpsyn':10, 'protein_RNAP_machinery':10, 'protein_Ribo_machinery':10,
     'protein_RNAase_machinery':10}
R = CRN3.simulate_with_bioscrape(timepoints, initial_condition_dict = x0)

/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/html5lib/_trie/_base.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
odeint failed with mxstep=500...

In [4]:
#R

In [7]:
palette = colorcet.b_glasbey_category10
palette.remove('#d62628')

In [6]:
p = bokeh.plotting.figure(width = 450, height = 340, 
                         x_axis_label = 'time (sec)', y_axis_label = 'concentration',
                         title = 'ATP synthase tx and tl')
plot_list = ['rna_atpsyn', 'protein_atpsyn', 'dna_atpsyn']
for i,item in enumerate(plot_list):
    p.line(timepoints, R[item], legend_label = item, color = palette[i], line_width = 2)
    
p.legend.location = 'center_right'
bokeh.io.show(p)

## SBML for ATP synthase membrane integration
---

In [19]:
atpsyn = Species("atpsyn", material_type = "protein")

atpsyn_b = Species("atpsyn_b", material_type = "protein")

k_bound = 1

R = Reaction([atpsyn], [atpsyn_b], k_bound)

CRN_mi = ChemicalReactionNetwork(species = [atpsyn, atpsyn_b], reactions = [R])

print(CRN_mi.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

Species (2) = {0. protein[atpsyn], 1. protein[atpsyn_b]}
Reactions (1) = [
0. protein[atpsyn] --> protein[atpsyn_b]        
        massaction: k_f(protein[atpsyn])=1*protein[atpsyn]
]


In [20]:
# Write SBML file
CRN_mi.write_sbml_file('atpsynthase_membrane_integration.xml')

True

In [27]:
timepoints = np.linspace(0,10,100)
x0_mi = {'protein_atpsyn':10}
R_mi = CRN_mi.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_mi)

In [28]:
R_mi

,protein_atpsyn,protein_atpsyn_b,time
0,10.000000,0.000000,0.00000
1,9.039239,0.960761,0.10101
2,8.170784,1.829216,0.20202
3,7.385767,2.614233,0.30303
4,6.676171,3.323829,0.40404
...,...,...,...
95,0.000680,9.999320,9.59596
96,0.000615,9.999385,9.69697
97,0.000556,9.999444,9.79798
98,0.000502,9.999498,9.89899


In [29]:
p_mi = bokeh.plotting.figure(width = 450, height = 340, 
                         x_axis_label = 'time (sec)', y_axis_label = 'concentration',
                         title = 'ATP synthase membrane integration')
plot_list = ['protein_atpsyn', 'protein_atpsyn_b']
for i,item in enumerate(plot_list):
    p_mi.line(timepoints, R_mi[item], legend_label = item, color = palette[i], line_width = 2)
    
p_mi.legend.location = 'center_right'
bokeh.io.show(p_mi)

## SBML for ATP Synthesis via ATP synthase membrane protein
---

In [10]:
outside_H = Species("outside_H", material_type = "proton")

inside_H = Species("inside_H", material_type = "proton")

atpsyn_b = Species("atpsyn_b", material_type = "protein")

adp = Species('adp', material_type = 'metabolite')
atp = Species('atp', material_type = 'metabolite')
pi = Species('pi', material_type = 'metabolite')


k_atp_synthesis = 2

R1 = Reaction([adp,pi, atpsyn_b], [atp], k_atp_synthesis)

R2 = Reaction([outside_H, outside_H, outside_H, outside_H], 
             [inside_H, inside_H, inside_H, inside_H], k_atp_synthesis)

CRN_atpsynthesis = ChemicalReactionNetwork(species = [outside_H, inside_H, atpsyn_b, adp, atp, pi], reactions = [R1,R2])

print(CRN_atpsynthesis.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

Species (6) = {0. proton[outside_H], 1. proton[inside_H], 2. protein[atpsyn_b], 3. metabolite[adp], 4. metabolite[atp], 5. metabolite[pi]}
Reactions (2) = [
0. metabolite[adp] + metabolite[pi] + protein[atpsyn_b] --> metabolite[atp]        
        massaction: k_f(metabolite[adp],metabolite[pi],protein[atpsyn_b])=2*metabolite[adp]*metabolite[pi]*protein[atpsyn_b]
1. 4 proton[outside_H] --> 4 proton[inside_H]        
        massaction: k_f(proton[outside_H])=2*proton[outside_H]^4
]


In [11]:
# Write SBML file
CRN_atpsynthesis.write_sbml_file('atp_synthesis_through_atpsynthase.xml')

True

In [60]:
timepoints = np.linspace(0,2,100)
x0_atpsynthesis = {'proton_outside_H':10, 'metabolite_adp':10, 'metabolite_pi':10}
R_atpsynthesis = CRN_atpsynthesis.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_atpsynthesis)

In [61]:
p_atpsynthesis = bokeh.plotting.figure(width = 450, height = 340, 
                         x_axis_label = 'time (sec)', y_axis_label = 'concentration',
                         title = 'ATP synthesis through ATP synthase')
plot_list = ['proton_outside_H', 'proton_inside_H', 'metabolite_atp']
for i,item in enumerate(plot_list):
    p_atpsynthesis.line(timepoints, R_atpsynthesis[item], legend_label = item, color = palette[i], line_width = 2)
    
p_atpsynthesis.legend.location = 'center_right'
bokeh.io.show(p_atpsynthesis)

## SBML for Proton Pump transcription and translation
---

In [46]:
G_pp = DNAassembly("proton_pump", promoter = "atpsyn_proton_pump", rbs = "proton_pumnp_rbs", transcript = None, protein = None)
M3_pp = TxTlDilutionMixture("liposome", components = [G_pp], parameter_file = "default_parameters.txt")
CRN3_pp = M3_pp.compile_crn()

#print(repr(M3),"\n", CRN3_pp.pretty_print(show_attributes = True, show_material = True, show_rates = True),"\n\n")



In [47]:
# Write SBML file
CRN3_pp.write_sbml_file('proton_pump_tx_tl.xml')

True

In [49]:
timepoints = np.linspace(0,10000,100)
x0_pp = {'dna_proton_pump':10, 'protein_RNAP_machinery':10, 'protein_Ribo_machinery':10,
     'protein_RNAase_machinery':10}
R_pp = CRN3_pp.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_pp)

/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
odeint failed with mxstep=500...

In [50]:
p_pp = bokeh.plotting.figure(width = 450, height = 340, 
                         x_axis_label = 'time (sec)', y_axis_label = 'concentration',
                         title = 'Proton pump tx and tl')
plot_list = ['rna_proton_pump', 'protein_proton_pump', 'dna_proton_pump']
for i,item in enumerate(plot_list):
    p_pp.line(timepoints, R_pp[item], legend_label = item, color = palette[i], line_width = 2)
    
p_pp.legend.location = 'center_right'
bokeh.io.show(p_pp)

## SBML for proton pump membrane integration
---

In [51]:
proton_pump = Species("proton_pump", material_type = "protein")

proton_pump_b = Species("proton_pump_b", material_type = "protein")

k_bound = 1

R_pp = Reaction([proton_pump], [proton_pump_b], k_bound)

CRN_mi_pp = ChemicalReactionNetwork(species = [proton_pump, proton_pump_b], reactions = [R_pp])

print(CRN_mi_pp.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

Species (2) = {0. protein[proton_pump], 1. protein[proton_pump_b]}
Reactions (1) = [
0. protein[proton_pump] --> protein[proton_pump_b]        
        massaction: k_f(protein[proton_pump])=1*protein[proton_pump]
]


In [53]:
# Write SBML file
CRN_mi_poutside_H = Species("outside_H", material_type = "proton")

inside_H = Species("inside_H", material_type = "proton")

adp = Species('adp', material_type = 'metabolite')
atp = Species('atp', material_type = 'metabolite')
pi = Species('pi', material_type = 'metabolite')


k_atp_synthesis = 2

R1 = Reaction([adp,pi], [atp], k_atp_synthesis)

R2 = Reaction([outside_H, outside_H, outside_H, outside_H], 
             [inside_H, inside_H, inside_H, inside_H], k_atp_synthesis)

CRN_atpsynthesis = ChemicalReactionNetwork(species = [outside_H, inside_H, adp, atp, pi], reactions = [R1,R2])

print(CRN_atpsynthesis.pretty_print(show_materials = True, show_rates = True, show_attributes = True))p.write_sbml_file('proton_pump_membrane_integration.xml')

True

In [55]:
timepoints = np.linspace(0,10,100)
x0_mi_pp = {'protein_proton_pump':10}
R_mi_pp = CRN_mi_pp.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_mi_pp)

In [56]:
p_mi_pp = bokeh.plotting.figure(width = 450, height = 340, 
                         x_axis_label = 'time (sec)', y_axis_label = 'concentration',
                         title = 'Proton pump membrane integration')
plot_list = ['protein_proton_pump', 'protein_proton_pump_b']
for i,item in enumerate(plot_list):
    p_mi_pp.line(timepoints, R_mi_pp[item], legend_label = item, color = palette[i], line_width = 2)
    
p_mi_pp.legend.location = 'center_right'
bokeh.io.show(p_mi_pp)

## SBML for proton transport through proton pump
---

In [62]:
outside_H = Species("outside_H", material_type = "proton")

inside_H = Species("inside_H", material_type = "proton")

adp = Species('adp', material_type = 'metabolite')
atp = Species('atp', material_type = 'metabolite')
pi = Species('pi', material_type = 'metabolite')


k_proton_out = 2

R1 = Reaction([adp,pi], [atp], k_atp_synthesis)

R2 = Reaction([inside_H, inside_H, inside_H, inside_H],
              [outside_H, outside_H, outside_H, outside_H], k_proton_out)

CRN_protontransport = ChemicalReactionNetwork(species = [outside_H, inside_H, adp, atp, pi], reactions = [R1,R2])

print(CRN_protontransport.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

Species (5) = {0. proton[outside_H], 1. proton[inside_H], 2. metabolite[adp], 3. metabolite[atp], 4. metabolite[pi]}
Reactions (2) = [
0. metabolite[adp] + metabolite[pi] --> metabolite[atp]        
        massaction: k_f(metabolite[adp],metabolite[pi])=2*metabolite[adp]*metabolite[pi]
1. 4 proton[inside_H] --> 4 proton[outside_H]        
        massaction: k_f(proton[inside_H])=2*proton[inside_H]^4
]


In [63]:
# Write SBML file
CRN_protontransport.write_sbml_file('proton_transport_through_proton_pump.xml')

True

In [64]:
timepoints = np.linspace(0,2,100)
x0_protontransport = {'proton_inside_H':10, 'metabolite_adp':10, 'metabolite_pi':10}
R_protontransport = CRN_protontransport.simulate_with_bioscrape(timepoints, 
                                                                initial_condition_dict = x0_protontransport)


In [65]:
p_protontransport = bokeh.plotting.figure(width = 450, height = 340, 
                         x_axis_label = 'time (sec)', y_axis_label = 'concentration',
                         title = 'Proton transport through proton pump')
plot_list = ['proton_outside_H', 'proton_inside_H', 'metabolite_atp']
for i,item in enumerate(plot_list):
    p_protontransport.line(timepoints, R_protontransport[item], legend_label = item, color = palette[i], line_width = 2)
    
p_protontransport.legend.location = 'center_right'
bokeh.io.show(p_protontransport)

## SBML for ATP Use

In [3]:
adp = Species('adp', material_type = 'metabolite')
atp = Species('atp', material_type = 'metabolite')
pi = Species('pi', material_type = 'metabolite')

k_atp_use = 5

R1_atpuse = Reaction([atp], [adp, pi], k_atp_use)


CRN_atpuse = ChemicalReactionNetwork(species = [adp, atp, pi], reactions = [R1_atpuse])

print(CRN_atpuse.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

Species (3) = {0. metabolite[adp], 1. metabolite[atp], 2. metabolite[pi]}
Reactions (1) = [
0. metabolite[atp] --> metabolite[adp] + metabolite[pi]        
        massaction: k_f(metabolite[atp])=5*metabolite[atp]
]


In [4]:
# Write SBML file
CRN_atpuse.write_sbml_file('atp_use.xml')

True

In [5]:
timepoints = np.linspace(0,2,100)
x0_atpuse = {'metabolite_atp':10}
R_atpuse = CRN_atpuse.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_atpuse)

In [8]:
p_atpuse = bokeh.plotting.figure(width = 450, height = 340, 
                         x_axis_label = 'time (sec)', y_axis_label = 'concentration',
                         title = 'ATP Use')
plot_list = ['metabolite_atp']
for i,item in enumerate(plot_list):
    p_atpuse.line(timepoints, R_atpuse[item], legend_label = item, color = palette[i], line_width = 2)
    
p_atpuse.legend.location = 'center_right'
bokeh.io.show(p_atpuse)